In [1]:
import pandas as pd

In [2]:
# Read in student and school data
student_data_original = "Resources/students_complete.csv"
school_data_original = "Resources/schools_complete.csv"

# Build csvreader for each file
student_data_df = pd.read_csv(student_data_original)
# student_data_df.head()

# Second csvreader
school_data_df = pd.read_csv(school_data_original)
# school_data_df

# Alphabatize the school dataframe
sorted_school_data = school_data_df.sort_values("school_name")
# sorted_school_data

# Merge data sets based on school name
combined_data = pd.merge(student_data_df, sorted_school_data, how="left", on=["school_name", "school_name"])
# combined_data.head()

In [3]:
# Distric Summary Calculations and dataframe construction

# Calculate total number of schools as length of list of unique schools. Remember that index starts at 0
total_schools = sorted_school_data["school_name"].unique()

# Calculate total number of students as length of list of unique student ids (names might be repeated)
student_total = combined_data["Student ID"].unique()
total_students = len(student_total)

# Calculate the total budget
total_budget = sum(school_data_df["budget"])

# Calculate the average math and reading score
average_math_score = combined_data["math_score"].mean()
average_reading_score = combined_data["reading_score"].mean()

# Calculate the precentage of students that pass math 
# Don't do a list, do .loc
passed_math_df = combined_data.loc[combined_data["math_score"] > 69, "Student ID"]
passed_math_count = len(passed_math_df)
passed_math_percent = (passed_math_count / total_students) * 100 

# Calculate percentage of students that passed reading

passed_reading_df = combined_data.loc[combined_data["reading_score"] > 69, "Student ID"]
passed_reading_count = len(passed_reading_df)
passed_reading_percent = (passed_reading_count / total_students) * 100

# Calculate the percentage of students that pass BOTH math and reading
# Merge the dataframes and only include students with data in both
# Should be an inner merge
passed_reading_and_math_df = pd.merge(passed_math_df, passed_reading_df, on="Student ID")

# Calculate percentage of students who passed both
passed_both_count = len(passed_reading_and_math_df)
passed_both_percent = (passed_both_count / total_students) * 100

In [4]:
# Create dataframe to hold all results, capitalization matters
District_Summary_df = pd.DataFrame({
    "Total Schools": [len(total_schools)], 
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "Percent Passing Math": [passed_math_percent],
    "Percent Passing Reading": [passed_reading_percent],
    "Percent Passing Overall":[passed_both_percent]
})

District_Summary_df = District_Summary_df.style.format({"Total Students": "{:,.0f}", "Total Budget":"${:,.0f}"})

District_Summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
0,15,"39,170","$24,649,428",78.985371,81.877840,74.980853,85.805463,65.172326


In [5]:
# Clean formatting (optional)
# Add commas? Other?

In [6]:
# School Summary Calculations and Dataframe construction

# Sort data to be alphabetical by school name
sorted_data = combined_data.sort_values("school_name")
# sorted_data

# Make dataframe with school name, type, budget, and size to start final summary
school_summary_df = sorted_school_data[["school_name", "type", "size", "budget"]]
# school_summary_df

school_summary_df["Per Student Budget"] = school_summary_df["budget"] / school_summary_df["size"]
# school_summary_df

# Calculate average reading and math scores

# Group sorted data by school name
sorted_grouped_data = sorted_data.groupby("school_name")
# Create list of student counts
student_count_list = sorted_grouped_data["Student ID"].count()

# Start with list of scores
reading_score_list = sorted_grouped_data["reading_score"].sum()
math_score_list = sorted_grouped_data["math_score"].sum()

# Divide total scores in reading and math by student_count
average_reading_score_by_school = reading_score_list / student_count_list
average_math_score_by_school = math_score_list / student_count_list


# Calculate the percent of students who passed math and those who passed reading

# Count those who passed for each school
# If this works, it is thanks to stack overflow geniuses, thank you stackoverflow!
# https://stackoverflow.com/questions/13851535/how-to-delete-rows-from-a-pandas-dataframe-based-on-a-conditional-expression
passing_math_score = sorted_data.drop(sorted_data[sorted_data.math_score < 70].index)
# passing_math_score.head()
passing_reading_score = sorted_data.drop(sorted_data[sorted_data.reading_score < 70].index)
# passing_reading_score.head()

# Group passing score dataframes by school
grouped_math = passing_math_score.groupby("school_name")
grouped_reading = passing_reading_score.groupby("school_name")

# Create list of student counts for math and reading passing
passed_math_count_by_school = grouped_math["Student ID"].count()
passed_reading_count_by_school = grouped_reading["Student ID"].count()

# Divide passing counts by total student counts and multiply by 100
passed_reading_percent = (passed_reading_count_by_school / student_count_list) * 100
# passed_reading_percent
passed_math_percent = (passed_math_count_by_school / student_count_list) * 100
# passed_math_percent

# Calculate the number of students who passed both math and reading
# Drop non-passing math scores from passing_reading_score dataframe

passed_overall = passing_reading_score.drop(passing_reading_score[passing_reading_score.math_score < 70].index)
grouped_pass_overall = passed_overall.groupby("school_name")
passed_overall_by_school = grouped_pass_overall["Student ID"].count()
passed_overall_percent = (passed_overall_by_school / student_count_list) * 100
# passed_overall_percent
                      

In [7]:
# Start score dataframe to merge with summary later
reading_math_df = pd.DataFrame({
    "Average Reading Score": average_reading_score_by_school,
    "Average Math Score": average_math_score_by_school,
    "Percent Passing Reading": passed_reading_percent,
    "Percent Passing Math": passed_math_percent,
    "Percent Passing Overall": passed_overall_percent
})
# reading_math_df

In [43]:
# Final School Summary
# having trouble renaming columns

# Merge reading_math_df with school_summary_df for school_summmary_final
# I think I'm having problems later because I added School_Summary_df and it's making trouble for me
# I'm merging each independently in the hopes that not have these two dataframes related will solve the issue
school_summary_final = pd.merge(school_summary_df, reading_math_df, how="left", on=["school_name", "school_name"])
# school_summary_final

School_Summary_df = pd.merge(school_summary_df, reading_math_df, how="left", on=["school_name", "school_name"])

School_Summary_df["budget"] = School_Summary_df["budget"].map("${:,.2f}".format)
School_Summary_df["Per Student Budget"] = School_Summary_df["Per Student Budget"].map("${:,.2f}".format)

School_Summary_df

,school_name,type,size,budget,Per Student Budget,Average Reading Score,Average Math Score,Percent Passing Reading,Percent Passing Math,Percent Passing Overall
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,81.033963,77.048432,81.933280,66.680064,54.642283
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.975780,83.061895,97.039828,94.133477,91.334769
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,81.158020,76.711767,80.739234,65.988471,53.204476
3,Ford High School,District,2739,"$1,763,916.00",$644.00,80.746258,77.102592,79.299014,68.309602,54.289887
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.816757,83.351499,97.138965,93.392371,90.599455
5,Hernandez High School,District,4635,"$3,022,020.00",$652.00,80.934412,77.289752,80.862999,66.752967,53.527508
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.814988,83.803279,96.252927,92.505855,89.227166
7,Huang High School,District,2917,"$1,910,635.00",$655.00,81.182722,76.629414,81.316421,65.683922,53.513884
8,Johnson High School,District,4761,"$3,094,650.00",$650.00,80.966394,77.072464,81.222432,66.057551,53.539172
9,Pena High School,Charter,962,"$585,858.00",$609.00,84.044699,83.839917,95.945946,94.594595,90.540541


In [44]:
# Clean up formatting in school_summary_final

In [45]:
# Top Schools Summary

# Sort schools by overall passing percent, reset index, and drop those below 5 for top 5 schools
schools_sort_top = school_summary_final.sort_values("Percent Passing Overall", ascending=False)
reindex_schools = schools_sort_top.reset_index(drop=True)
top_schools = reindex_schools.drop(reindex_schools[reindex_schools.index > 4].index)

top_schools["budget"] = top_schools["budget"].map("${:,.2f}".format)
top_schools["Per Student Budget"] = top_schools["Per Student Budget"].map("${:,.2f}".format)

top_schools

,school_name,type,size,budget,Per Student Budget,Average Reading Score,Average Math Score,Percent Passing Reading,Percent Passing Math,Percent Passing Overall
0,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.975780,83.061895,97.039828,94.133477,91.334769
1,Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.848930,83.418349,97.308869,93.272171,90.948012
2,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.816757,83.351499,97.138965,93.392371,90.599455
3,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.989488,83.274201,96.539641,93.867718,90.582567
4,Pena High School,Charter,962,"$585,858.00",$609.00,84.044699,83.839917,95.945946,94.594595,90.540541


In [46]:
# Bottom Schools Summary

# Sort schools by overall passing percent, reset index, and drop those below 5 for bottom 5 schools
schools_sort_bottom = school_summary_final.sort_values("Percent Passing Overall", ascending=True)
reindex_schools = schools_sort_bottom.reset_index(drop=True)
bottom_schools = reindex_schools.drop(reindex_schools[reindex_schools.index > 4].index)

bottom_schools["budget"] = bottom_schools["budget"].map("${:,.2f}".format)
bottom_schools["Per Student Budget"] = bottom_schools["Per Student Budget"].map("${:,.2f}".format)

bottom_schools

,school_name,type,size,budget,Per Student Budget,Average Reading Score,Average Math Score,Percent Passing Reading,Percent Passing Math,Percent Passing Overall
0,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,80.744686,76.842711,80.220055,66.366592,52.988247
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,81.158020,76.711767,80.739234,65.988471,53.204476
2,Huang High School,District,2917,"$1,910,635.00",$655.00,81.182722,76.629414,81.316421,65.683922,53.513884
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,80.934412,77.289752,80.862999,66.752967,53.527508
4,Johnson High School,District,4761,"$3,094,650.00",$650.00,80.966394,77.072464,81.222432,66.057551,53.539172


In [47]:
# Clean up formatting

In [48]:
# Math Scores by Grade Summary
# Ask Kelly about if there is a better way
# What type of conditional would let me do this all at once?

In [49]:
# Call student_data_df to verify needed info
student_data_df

# Separate each grade students
ninth_grade = student_data_df.drop(student_data_df[student_data_df.grade != "9th"].index)
tenth_grade = student_data_df.drop(student_data_df[student_data_df.grade != "10th"].index)
eleven_grade = student_data_df.drop(student_data_df[student_data_df.grade != "11th"].index)
twelve_grade = student_data_df.drop(student_data_df[student_data_df.grade != "12th"].index)

# Try with groupby
group_ninth = ninth_grade.groupby("school_name")
group_tenth = tenth_grade.groupby("school_name")
group_eleven = eleven_grade.groupby("school_name")
group_twelve = twelve_grade.groupby("school_name")

# Find grade size
ninth_class_size = group_ninth["Student ID"].count()
tenth_class_size = group_tenth["Student ID"].count()
eleven_class_size = group_eleven["Student ID"].count()
twelve_class_size = group_twelve["Student ID"].count()

# Hold math totals
ninth_math_total = group_ninth["math_score"].sum()
tenth_math_total = group_tenth["math_score"].sum()
eleven_math_total = group_eleven["math_score"].sum()
twelve_math_total = group_twelve["math_score"].sum()

# Average math score list for each grade
ninth_math_avg = ninth_math_total / ninth_class_size
tenth_math_avg = tenth_math_total / tenth_class_size
eleven_math_avg = eleven_math_total / eleven_class_size
twelve_math_avg = twelve_math_total / twelve_class_size

In [50]:
# Create dataframe to hold summary
math_score_summary = pd.DataFrame({
    "9th": ninth_math_avg,
    "10th": tenth_math_avg,
    "11th": eleven_math_avg,
    "12th": twelve_math_avg
})
math_score_summary

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [51]:
# Clean up formatting

In [52]:
# Reading Scores by Grade Summary

# Using Grade Groups and class sizes from the math scores, calculate reading averages by grade

# Hold reading totals
ninth_reading_total = group_ninth["reading_score"].sum()
tenth_reading_total = group_tenth["reading_score"].sum()
eleven_reading_total = group_eleven["reading_score"].sum()
twelve_reading_total = group_twelve["reading_score"].sum()

# Average reading score list for each grade
ninth_reading_avg = ninth_reading_total / ninth_class_size
tenth_reading_avg = tenth_reading_total / tenth_class_size
eleven_reading_avg = eleven_reading_total / eleven_class_size
twelve_reading_avg = twelve_reading_total / twelve_class_size

In [53]:
# Create dataframe to hold summary
reading_score_summary = pd.DataFrame({
    "9th": ninth_reading_avg,
    "10th": tenth_reading_avg,
    "11th": eleven_reading_avg,
    "12th": twelve_reading_avg
})
reading_score_summary

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [54]:
# Clean Formatting

In [55]:
# Average Math and Reading Scores by
# Per Student Spending, School Size, and School Type
# Will all use school_summary_final as source dataframe

In [57]:
# Scores by Per Student Spending
# Fixed this cell by untangling school_summary_final and School_Summary_df earlier

# Make bins and name groups to apply to school_summary_final
spending_bins = [0, 585, 630, 645, 680]
spending_ranges_per_student = ["<$585", "$585-630", "$630-645", "$645-680"]

# Cut the data, groupby spending ranges, make new df, and drop columns
school_summary_final["Spending Ranges Per Student"] = pd.cut(school_summary_final["Per Student Budget"], spending_bins, labels=spending_ranges_per_student, include_lowest=True)
grouped_spending_ranges = school_summary_final.groupby("Spending Ranges Per Student")
scores_by_school_spending = grouped_spending_ranges.mean()
scores_by_school_spending = scores_by_school_spending.drop(["size", "budget", "Per Student Budget"], axis=1)
scores_by_school_spending


,Average Reading Score,Average Math Score,Percent Passing Reading,Percent Passing Math,Percent Passing Overall
Spending Ranges Per Student,,,,,
<$585,83.933814,83.455399,96.610877,93.460096,90.369459
$585-630,83.155286,81.899826,92.718205,87.133538,81.418596
$630-645,81.624473,78.518855,84.391793,73.484209,62.857656
$645-680,81.027843,76.997210,81.133951,66.164813,53.526855


In [58]:
# Scores by school size

# Set bins and labels
size_bins = [0, 1000, 2000, 5000]
size_ranges = ["Small(<1000)", "Medium(1000-2000)", "Large(2000-5000)"]

# Cut the data, groupby size ranges, set new df and drop columns
school_summary_final["School Size"] = pd.cut(school_summary_final["size"], size_bins, labels=size_ranges, include_lowest=True)
grouped_size_ranges = school_summary_final.groupby("School Size")
scores_by_school_size = grouped_size_ranges.mean()
scores_by_school_size = scores_by_school_size.drop(["size", "budget", "Per Student Budget"], axis=1)
scores_by_school_size


,Average Reading Score,Average Math Score,Percent Passing Reading,Percent Passing Math,Percent Passing Overall
School Size,,,,,
Small(<1000),83.929843,83.821598,96.099437,93.550225,89.883853
Medium(1000-2000),83.864438,83.374684,96.790680,93.599695,90.621535
Large(2000-5000),81.344493,77.746417,82.766634,69.963361,58.286003


In [ ]:
# Scores by school type

# Groupby type, save as new df and drop unwanted columns
grouped_by_type = school_summary_final.groupby("type")
scores_by_school_type = grouped_by_type.mean()
scores_by_school_type = scores_by_school_type.drop(["size", "budget", "Per Student Budget"], axis=1)
scores_by_school_type

#rename type column to Type

In [ ]:
# Everything below is a leftover

In [ ]:
# Scores by school type

# Groupby type
grouped_by_type = school_summary_final.groupby("type")
grouped_by_type.mean()

# Make new dataframe to modify as needed
scores_by_school_type = grouped_by_type.mean()
scores_by_school_type

scores_by_school_type = scores_by_school_type.drop(["size", "budget", "Per Student Budget"], axis=1)
scores_by_school_type

In [ ]:
# Interesting breadcrumbs from meeting with Kelly
# grouped_spending_ranges

# dir(grouped_spending_ranges)

# pd.DataFrame